# Introduction
In this kernelI tried to train my model and get the best accuracy as I can. I tried different ephocs and different activation styles, after that I ended up with this values below.

1. [Loading Train and Test Data](#1)
1. [Arranging Train Data](#2)
1. [Traning Model](#3)
1. [Fitting Model on Train Data](#4)
1. [Arranging Test Data](#5)
1. [Fitting Model on Test Data](#6)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt



import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"><br></a>
## Loading Train and Test Data

In [ ]:
train=pd.read_csv("/kaggle/input/mnist-in-csv/mnist_train.csv")
train.head()

As we can see from the table above, we can easily say that every photo of digits made from 28x28 pixels and the first column of the data is represents the digit.

In [ ]:
test=pd.read_csv("/kaggle/input/mnist-in-csv/mnist_test.csv")

test.head()

According to our expectations we have same condition like in the train data frame.

<a id="2"><br></a>
## Arranging Train Data

Now for seeing a sample inside the train we are going to make one of the pics as a matrix. After that reshape it and print it.

In [ ]:
print(train.iloc[15][0]) #this is for seeing which number we are picturing.
img=train.iloc[15][1:].to_numpy() #you can also use as_matrix(), but the last time that I wanted to use it I've got a problem. Because of that, I used to_numpy(). 
                                  #I didn't get rid of the label part yet. Fot just using pixels we started from the 2nd([1]) element of one column.
img=np.reshape(img,(28,28))
plt.imshow(img,cmap="gray")
plt.axis("off")
plt.show()

Now we are going to seperate our labels from pixels.

In [ ]:
X_train=train.drop("label",axis=1)
X_train.head()

In [ ]:
Y_train=train.label
Y_train.head()

Now we can easily see how many samples do we have from each number.

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(Y_train,palette="rocket")
plt.title("Count of Numbers")
plt.show()


Now we are going to initialize our pixel values

In [ ]:
X_train=X_train/255.0
X_train.shape

In [ ]:
Y_train.shape

For making our train frames work with keras, we need to reshape them.

In [ ]:
X_train=X_train.values.reshape(-1,28,28,1)
X_train.shape

In [ ]:
from keras.utils.np_utils import to_categorical
Y_train=to_categorical(Y_train,num_classes=10)
Y_train.shape

After reshaping train data frames, now we can seperate them as train and validation.

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(X_train,Y_train,test_size=0.15,random_state=42)

print("x train shape: ",x_train.shape)
print("x val shape: ",x_val.shape)
print("y train shape: ",y_train.shape)
print("y val shape: ",y_val.shape)

From now on we are starting to train model.

<a id="3"><br></a>
## Training Model

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,Activation
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(28,28,1)))
model.add(Activation("tanh"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3)))
model.add(Activation("tanh"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3)))
model.add(Activation("tanh"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256,activation="tanh"))
model.add(Dropout(0.5))
model.add(Dense(10,activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [ ]:
datagen=ImageDataGenerator(rotation_range=0.5,zoom_range=0.5,horizontal_flip=True)
datagen.fit(x_train)

<a id="4"><br></a>
## Fitting Model on Train Data

In [ ]:
history=model.fit_generator(datagen.flow(x_train,y_train,batch_size=32),epochs=50,validation_data=(x_val,y_val), steps_per_epoch=x_train.shape[0] // 32)

In [ ]:
plt.subplot(1,2,1)
plt.plot(history.history["val_loss"],color="r",label="validation loss")
plt.xlabel("number of ephocs")
plt.ylabel("val_loss")

plt.subplot(1,2,2)
plt.plot(history.history["val_accuracy"],color="r",label="validation accuracy")
plt.xlabel("number of ephocs")
plt.ylabel("val_accuracy")

plt.show()

plt.subplot(1,2,1)
plt.plot(history.history["loss"],color="r",label="loss")
plt.xlabel("number of ephocs")
plt.ylabel("loss")

plt.subplot(1,2,2)
plt.plot(history.history["accuracy"],color="r",label="accuracy")
plt.xlabel("number of ephocs")
plt.ylabel("accuracy")

plt.show()

In [ ]:
Y_pred = model.predict(x_val)

Y_pred_classes = np.argmax(Y_pred,axis = 1) 

Y_true = np.argmax(y_val,axis = 1) 

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

<a id="5"><br></a>
## Arranging Test Data

In [ ]:
x_test=test.drop("label",axis=1)

x_test.head()

In [ ]:
y_test=test.label

y_test.head()

In [ ]:
x_test=x_test/255.0
x_test=x_test.values.reshape(-1,28,28,1)

x_test.shape

In [ ]:
y_test=to_categorical(y_test,num_classes=10)

y_test.shape

<a id="6"><br></a>
## Fitting Model on Test

In [ ]:
Y_pred = model.predict(x_test)
 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 

Y_true = np.argmax(y_test,axis = 1) 

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()